# Введение в анализ данных
## НИУ ВШЭ, 2020-2021 учебный год

### Домашнее задание №8

In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, mean_squared_error

## Градиентный бустинг своими руками (2 + 2* баллов)

Реализуйте алгоритм градиентного бустинга для регрессии. Напомним основные формулы.

Обозначим текущую композицию на $N-1$ шаге за $a_{N - 1}(x_i)$. Следующий базовый алгоритм $b_N(x_i)$ обучается на ответах $-\frac{\partial L(y_i, z)}{\partial z}\Bigl|_{z = a_{N - 1}(x_i)}$, где $L(y_i, z)$ — значение функции потерь на объекте при правильном ответе $y_i$ и предсказании $z$. Композиция на следующем шаге получается следующим образом:

$$
a_N(x_i) = a_{N-1}(x_i) + \nu\gamma_Nb_N(x_i)
$$

Здесь $\nu$ — гиперпараметр learning rate, $\gamma_N$ — оптимальный вес, настраиваемый на каждом шаге алгоритма, который можно найти по следующей формуле (обратите внимание на отсутствие $\nu$):

$$
\gamma_N = \mathrm{arg}\min_\gamma \frac{1}{\ell}\sum\limits_{i=1}^{\ell}L\left(y_i, a_{N - 1}(x_i) + \gamma b_N(x_i)\right)
$$

Можете принять $\gamma_N = 1$ для каждого $N$. Реализуйте нахождение оптимального $\gamma_N$ на каждом шаге, чтобы получить ещё 1 балл.

В качестве функции потерь возьмите MSE.

*Примечание. Вы можете использовать `DecisionTree` из `sklearn` и методы оптимизации из различных библиотек.*

In [2]:
# это может помочь вам для поиска оптимальных gamma
from scipy.optimize import minimize

In [3]:
class GradientBoosting(BaseEstimator):
    def __init__(self, n_estimators, max_depth, learning_rate=0.1):
        """
        PARAMETERS:
        n_estimators - number of trees in the ensemble
        max_depth - maximum depth of a tree
        learning_rate - coefficient by which new algorithm result is multiplied
        """
        # your code here
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        
        self.ensemble = []
        self.gammas = []
        pass
    
    def _loss_function(self, y, y_pred):
        return (y - y_pred)**2
    
    def _loss_gradient(self, y, y_pred):
        return -2 * (y - y_pred)
        
    def fit(self, x, y):
        """
        INPUT:
        x - np.array of shape (k, d)
        y - np.array of shape (k,)
        """
        # Здесь нам нужно проитерироваться по n_estimators и обучить
        # соответствующее количество деревьев с помощью _fit_predict_tree(),
        # правильно обновляя y_new
        # Деревья нужно где-то сохранить, чтобы затем использовать в predict()
        # your code here
        
        self.first_prediction_constant = np.mean(y)
        y_new = self.first_prediction_constant * np.ones(len(y))
        
        for i in range(self.n_estimators):
            y_new += self._fit_predict_tree(x, y, y_new)
            # your code here
        
        return self

    def _fit_predict_tree(self, x, y, y_new):
        """
        INPUT:
        x - np.array of shape (k, d)
        таргеты
        y - np.array of shape (k,)
        предсказания текущего ансамбля
        y_new - np.array of shape (k,)
        """
        # Обучаем дерево и возвращаем его предикшн
        y_antigradient = np.array([
            -1*self._loss_gradient(i, j) for i, j in zip(y, y_new)
        ])
        tree = DecisionTreeRegressor(max_depth=self.max_depth)
        tree = tree.fit(x, y_antigradient)
        self.ensemble.append(tree)
        y_pred = tree.predict(x)
        
        # Поиск оптимального lr на текущем шаге
        loss_by_lr = lambda lr : np.mean(self._loss_function(y, y_new + lr*y_pred))
        optimization_res  = minimize(loss_by_lr, 1)
        gamma = optimization_res['x'][0]
        self.gammas.append(gamma)
        
        # your code here
        return gamma * self.learning_rate * y_pred
        
    def predict(self, x):
        """
        INPUT:
        x - np.array of shape (m, d)
        OUTPUT:
        y_pred - np.array of shape (m,)
        """
        # Используем сохранённые деревья для расчёта агрегированного предикшна
        # your code here
        y_pred = self.first_prediction_constant * np.ones(x.shape[0])
        
        for i in range(self.n_estimators):
            y_pred += self.gammas[i]*self.learning_rate*self.ensemble[i].predict(x)
        
        return y_pred

Проверьте вашу реализацию на Boston dataset. Подберите оптимальные гиперпараметры, чтобы победить RandomForestRegressor как в обычном случае, так и при нахождении оптимального шага **(не меняйте параметры сида)**. При необходимости воспользуйтесь GridSearch. За это вы получите ещё 1 балл.

In [4]:
boston = load_boston()
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [5]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=4, n_estimators=640, random_state=19052019)

rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test))

9.63198271791959

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit

split_index = [-1]*len(X_train) + [0]*len(X_test)
pds = PredefinedSplit(test_fold = split_index)
np.random.seed(19052019)
g_search = GridSearchCV(
    GradientBoosting(400, 7),
    param_grid={
          'n_estimators' : [200, 250, 275, 300, 350, 400, 500, 600]
        , 'max_depth' : [6, 7, 8, 10]
        , 'learning_rate' : [0.1, 0.2, 0.25, 0.3]
    },
    cv=pds,
    scoring='neg_mean_squared_error',
    verbose=2,
    n_jobs=4
)

X, y = np.concatenate([X_train, X_test]), np.concatenate([y_train, y_test])
g_search.fit(X, y)

Fitting 1 folds for each of 128 candidates, totalling 128 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    9.7s
[Parallel(n_jobs=4)]: Done 128 out of 128 | elapsed:   31.8s finished


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=GradientBoosting(max_depth=7, n_estimators=400),
             n_jobs=4,
             param_grid={'learning_rate': [0.1, 0.2, 0.25, 0.3],
                         'max_depth': [6, 7, 8, 10],
                         'n_estimators': [200, 250, 275, 300, 350, 400, 500,
                                          600]},
             scoring='neg_mean_squared_error', verbose=2)

In [7]:
g_search.best_params_

{'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 400}

In [8]:
np.random.seed(19052019)
my_gb = GradientBoosting(
    n_estimators=g_search.best_params_['n_estimators'],
    max_depth=g_search.best_params_['max_depth'],
    learning_rate=g_search.best_params_['learning_rate']
).fit(X_train, y_train)

In [9]:
mean_squared_error(y_test, my_gb.predict(X_test))

11.604845701090515

Для удобства проверяющего укажите, пожалуйста, реализовали ли вы нахождение оптимального шага?

- Да

## Прогнозируем задержки самолётов (2 балла)

Поработаем с задачей про задержки самолётов. На основании доступных данных о рейсе вам нужно определить, будет ли он задержан на 15 минут.
Воспользуйтесь любыми методами градиентного бустинга {XGboost, catboost, LightGBM} и GridSearch для достижения результата. Получите 1 балл за преодоление порога roc_auc_score 0.72 и ещё 1 балл за преодоление порога 0.74.

In [10]:
from catboost import CatBoostClassifier

In [11]:
!curl -o flight_delays_train.csv https://raw.githubusercontent.com/PersDep/data-mining-intro-2021/main/hw08-boosting-clustering/flight_delays_train.csv
!curl -o flight_delays_test.csv https://raw.githubusercontent.com/PersDep/data-mining-intro-2021/main/hw08-boosting-clustering/flight_delays_test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     30      0 --:--:-- --:--:-- --:--:--    30
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     43      0 --:--:-- --:--:-- --:--:--    43


In [12]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [13]:
train.head()

,404: Not Found


In [14]:
train['day_of_month_sin'] = train['DayofMonth'].apply(lambda x : np.sin(2*np.pi*float(x[2:]))/31)
train['day_of_month_cos'] = train['DayofMonth'].apply(lambda x : np.cos(2*np.pi*float(x[2:]))/31)

KeyError: 'DayofMonth'

In [ ]:
train['day_of_week_sin'] = train['DayOfWeek'].apply(lambda x : np.sin(2*np.pi*float(x[2:]))/7)
train['day_of_week_cos'] = train['DayOfWeek'].apply(lambda x : np.cos(2*np.pi*float(x[2:]))/7)

In [ ]:
train['month_sin'] = train['Month'].apply(lambda x : np.sin(2*np.pi*float(x[2:]))/12)
train['month_cos'] = train['Month'].apply(lambda x : np.cos(2*np.pi*float(x[2:]))/12)

In [ ]:
train_feats = [
    'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance', 
    'day_of_month_sin', 'day_of_month_cos', 'day_of_week_sin', 'day_of_week_cos',
    'month_sin', 'month_cos', 'Month', 'DayOfWeek'
]

cat_feats = ['UniqueCarrier', 'Origin', 'Dest', 'Month', 'DayOfWeek']

In [ ]:
X_train = train[train_feats]
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3)

catbst = CatBoostClassifier()
catbst.fit(X_train_part, y_train_part, cat_feats)

In [ ]:
roc_auc_score(y_valid, catbst.predict_proba(X_valid)[:, 1])

## DBSCAN своими руками (2 + 2* баллов)

DBSCAN: Density-based spatial clustering of applications with noise

Реализуйте класс dbscan.

Алгоритм имеет два основных гиперпараметра:
1. `eps` &mdash; радиус рассматриваемой окрестности
2. `min_samples` &mdash; число соседей в окрестности

Для выполнения кластеризации DBSCAN точки делятся на основные точки, достижимые по плотности точки и выпадающие следующим образом:

- Точка $p$ является основной точкой, если по меньшей мере `min_samples` точек находятся на расстоянии, не превосходящем 
`eps` от неё. Говорят, что эти точки достижимы прямо из $p$.

-  Точка $q$ прямо достижима из $p$, если точка $q$ находится на расстоянии, не большем `eps`, от точки $p$, и $p$ — основная точка.
Точка $q$ достижима из $p$, если имеется путь 
$p_1,…,p_n$ где $p_1=p$ и $p_n=q$ , а каждая точка $p_{i+1}$ достижима прямо из $p_i$ (все точки на пути должны быть основными, за исключением $q$).

Все точки, не достижимые из основных точек, считаются выбросами.

Теперь, если $p$ является основной точкой, то она формирует кластер вместе со всеми точками (основными или неосновными), достижимыми из этой точки. Каждый кластер содержит по меньшей мере одну основную точку. Неосновные точки могут быть частью кластера, но они формируют его «край», поскольку не могут быть использованы для достижения других точек.


In [ ]:
# Это может помочь вам для поиска дистанции между точками
from scipy.spatial import distance

In [ ]:
# Рекомендации -- это только рекомендации
class dbscan:
    TYPE_CORE_POINT=0
    TYPE_BORDER_POINT=1
    TYPE_NOISE=2
    
    def __init__(self, eps, min_samples):
        # your code here
        self.eps = eps
        self.min_samples = min_samples
        
        pass
        
    def fit_predict(self, X):
        # Помечаем переданные точки, чтобы отличать уже кластеризованные
        # или являющиеся шумом от остальных
        # your code here
        self.points = X
        self.is_visited = np.array([False for i in range(X.shape[0])])
        self.cluster = np.array([-1 for i in range(X.shape[0])])
        
        # Счетчик кластеров
        C = -1
        
        # Реализуем цикл по точкам, игнорирущий уже кластеризованные
        # your code here
        for x_idx in range(self.points.shape[0]):
            if self.is_visited[x_idx] == False:
                # Если точка ещё не кластеризована, с помощью range_query() ищем её соседей
                # Если соседей недостаточно, то помечаем её выделенным под шум лейблом
                # (однако она ещё может попасть в какой-то иной кластер)
                # Если же соседей достаточно, будем строить от неё кластер с помощью grow_cluster()
                # your code here
                self.is_visited[x_idx] = True
                nbrs = self.range_query(x_idx).tolist()
                if len(nbrs) >= self.min_samples:
                    C += 1
                    self.grow_cluster(x_idx, nbrs, C)
        
        return self.cluster
                

    def grow_cluster(self, root_idx, neighbours, C):
        # В этой функции мы полностью строим кластер относительно переданной точки
        # Т.е. реализуем цикл по структуре данных, изначально состоящей из уже
        # известных соседей (neighbours) и в процессе пополняющейся
        # "соседями соседей" с помощью range_query()
        # your code here
        self.cluster[root_idx] = C
        for nbr in neighbours:
            if self.is_visited[nbr] == False:
                self.is_visited[nbr] = True
                nbrs_of_nbr = self.range_query(nbr).tolist()
                if len(nbrs_of_nbr) >= self.min_samples:
                    neighbours.extend(nbrs_of_nbr)
            if self.cluster[nbr] == -1:
                self.cluster[nbr] = C
                
        pass
            
        
        

    def range_query(self, x_idx):
        """Возвращает точки на расстоянии <= eps от x"""
        # С помощью функции dist()
        # your code here
        dists = self.dist(x_idx)
        nbrs = np.nonzero(dists <= self.eps)[0]
        
        return nbrs
    
    def dist(self, x_idx):
        """
        Метрика расстояния
        Считает расстояние между точкой под индексом x_idx и всеми точками в self.points
        """
        x = self.points[x_idx].reshape(1, -1)
        dists = distance.cdist(x, self.points, metric='euclidean').flatten()
        
        return dists

In [ ]:
x = [[1, 2, 3], [3, 4, 5]]
print(distance.pdist(x))

Проверим себя на простом примере:

In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import pandas as pd

X, _ = make_moons(n_samples=500, noise=0.1)
plt.scatter(X[:,0], X[:,1])
plt.show()

Кластеризуйте это с помощью своей версии DBSCAN:

In [ ]:
clusters = dbscan(eps=0.2, min_samples=10).fit_predict(X)

plt.scatter(X[:,0], X[:,1], c=clusters)
plt.show()

Референс:

In [ ]:
from sklearn.cluster import DBSCAN

clusters = DBSCAN(eps=0.2, min_samples=10).fit_predict(X)

plt.scatter(X[:,0], X[:,1], c=clusters)
plt.show()

Получите 1 балл за ещё один интересный пример кластеризации (в качестве референса используйте предоставленное изображение. Нужно несколько интересных соседствующих форм):

In [ ]:
from sklearn import datasets
X, _ = datasets.make_circles(n_samples=10000, noise=0.03, factor=.2)
X = X*25
B, _ = datasets.make_blobs(centers=[[-10, 8], [10, 8], [0, -15]], n_samples=500, cluster_std=0.5)
X = np.concatenate([X, B])
clusters = DBSCAN(eps=1, min_samples=2).fit_predict(X)

plt.scatter(X[:,0], X[:,1], c=clusters)
plt.show()

In [ ]:
clusters = dbscan(eps=1, min_samples=2).fit_predict(X)

plt.scatter(X[:,0], X[:,1], c=clusters)
plt.show()

Или хотя бы что-нибудь такое: 

Получите ещё 1 балл, если время работы вашего алгоритма больше референсного не более чем в 100 раз с учётом погрешности 3% от вашего времени:

In [ ]:
%%time
%%capture
clusters = DBSCAN(eps=0.2, min_samples=10).fit_predict(X)

In [ ]:
%%time
%%capture
clusters = dbscan(eps=0.2, min_samples=10).fit_predict(X)